In [ ]:
using GeoDataFrames
using DataFrames
using Plots
using Shapefile
using Random
using ArchGDAL
using GeoInterface
using ColorSchemes

# Set a backend for Plots.jl
gr()

In [ ]:
### Municipalities Shapefile
shapefile_path_municipalities = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Análises\\MT_Municipios_2022\\MT_Municipios_2022.shp"
municipalities = Shapefile.Table(shapefile_path_municipalities) |> DataFrame

# Highways Shapefile
shapefile_path_highways = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Análises\\Rodovias\\SNV_202410A.shp"
highways = Shapefile.Table(shapefile_path_highways) |> DataFrame

In [ ]:
# Filter for highways where sg_uf equals "MT"
filtered_highways_MT = filter(row -> row.sg_uf == "MT", highways)

# Group highways by the 'vl_br' column
grouped_highways = groupby(filtered_highways_MT, :vl_br)

In [ ]:
# Function to combine geometries within a group
function combine_geometries(geometries)
    # Filter out any missing or invalid geometries before processing
    valid_geoms = filter(g -> !isnothing(g) && GeoInterface.npoint(g) > 0, geometries)
    if isempty(valid_geoms)
        return nothing
    end
    
    return mapfoldl(
        x -> GeoInterface.convert(ArchGDAL, x), # map: convert each geometry to an ArchGDAL object
        ArchGDAL.union,                          # reduce: combine two geometries
        valid_geoms                              # the collection to operate on
    )
end

# Apply the function to create a new DataFrame with combined geometries
combined_highways = combine(grouped_highways, :geometry => combine_geometries => :geometry)

In [ ]:
## --- Plotting Section ---

# 1. Get the number of unique highways for coloring
num_highways = nrow(combined_highways)

# 2. Generate a list of high-contrast colors
palette = colorschemes[:Set1_9]
high_contrast_colors = [palette[mod1(i, length(palette))] for i in 1:num_highways]

# 3. Initialize the plot with a specific size
plot(size=(1000, 800), dpi=1800)

# 4. Plot the municipalities as the base map layer
plot!(municipalities.geometry,
    color=:white,
    linecolor=:black,
    label=false
)

# 5. Loop through the combined highways and plot each one
for (i, row) in enumerate(eachrow(combined_highways))
    plot!(row.geometry,
        color = high_contrast_colors[i], 
        linewidth = 4,
        label = "BR-" * string(row.vl_br)
    )
end

# 6. Finalize plot aesthetics
plot!(
    legend = :outerright,
    legendfontsize = 8,
    legendtitle = "Highways",
    framestyle = :none,
    grid = false
)

# 7. Display the plot on the screen 🖥️
display(current())

# 8. Save the final plot to a file 
Plots.savefig("C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Dissertação\\Quadros e gráficos\\Graph-Highways")